In [30]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State
import base64

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

from dbCRUD import DatabaseCRUD


###########################

# Data Manipulation / Model

###########################

username = "myUserAdmin"
password = "myPassword"
shelter = DatabaseCRUD(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))



#########################

# Dashboard Layout / View

#########################
                               
app = JupyterDash('Grazioso Salvare Animal Shelter Database')

image_filename = 'Grazioso_Salvare_Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Center(
        html.Img(
        # Style the image so it is only 300x150 pixels
        style={
            "width": 300,
            "height": 150
        },
        src='data:image/png;base64,{}'.format(encoded_image.decode())
        )
    ),
    html.Br(),
    html.Center(html.B(html.H1('Trevor Leon - Grazioso Salvare Dashboard'))),
    html.Hr(),
    # Here are the radio buttons and filter-type input variable styled in a row/horizontally
    dcc.RadioItems(
        id="filter-type",
        options=[
            {"label": "None", "value": "None"},
            {"label": "Water Rescue", "value": "Water Rescue"},
            {"label": "Mountain/Wilderness Rescue", "value": "Mountain/Wilderness Rescue"},
            {"label": "Disaster/Individual Tracking", "value": "Disaster/Individual Tracking"}
        ],
        value="None",
        labelStyle={"display": "inline-block"}
    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": True, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=True,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable="multi",
         # This line needs commented out, else the radio buttons don't select correctly for some reason.
        #row_selectable="multi",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10,
        hidden_columns=["1", "animal_id"]
        
    ),
    html.Br(),
    html.Hr(),
                               
    #This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
              style={'display' : 'flex'},
              children=[
                  html.Div(
                      id='graph-id',
                      className='col s12 m6',
                  ),
                  html.Div(
                      id='map-id',
                      className='col s12 m6',
                  )
              ]
             )
]
)

#############################################
                               
# Interaction Between Components / Controller
                               
#############################################
    
@app.callback(
    [Output('datatable-id','data'),
     Output('datatable-id','columns')],
    [Input('filter-type', 'value')])
def update_dashboard(filter_type):
        # If the filter-type is "None", read all from the shelter database.
        # Else, filter the search results so only desired animals are shown.
        if(filter_type == "None"):
            df = pd.DataFrame.from_records(shelter.read({}))
                               
        elif(filter_type == "Water Rescue"):
                               df = pd.DataFrame.from_records(
                                   shelter.read({
                                       "breed": [
                                           "Labrador Retriever Mix",
                                           "Chesapeak Bay Retriever",
                                           "Newfoundland"
                                       ],
                                       "sex_upon_outcome": "Intact Female",
                                       "age_upon_outcome_in_weeks": {"$gte": 26},
                                       "age_upon_outcome_in_weeks": {"$lte": 156}})
                               )
                
        elif(filter_type == "Mountain/Wilderness Rescue"):
                               df = pd.DataFrame.from_records(
                                  shelter.read({
                                      "breed": [
                                          "German Shepherd",
                                          "Alaskan Malamute",
                                          "Old English Sheepdog",
                                          "Siberian Husky",
                                          "Rottweiler"],
                                      "sex_upon_outcome": "Intact Male",
                                      "age_upon_outcome_in_weeks": {"$gte": 26},
                                      "age_upon_outcome_in_weeks": {"$lte": 156}})
                               )
                
        elif(filter_type == "Disaster/Individual Tracking"):
                               df = pd.DataFrame.from_records(
                                   shelter.read({
                                       "breed": [
                                           "Doberman Pinscher",
                                           "German Shepherd",
                                           "Golden Retriever",
                                           "Bloodhound",
                                           "Rottweiler"
                                       ],
                                       "sex_upon_outcome": "Intact Male",
                                       "age_upon_outcome_in_weeks": {"$gte": 20},
                                       "age_upon_outcome_in_weeks": {"$lte": 300}})
                               )
                               
        # Set the DataFrame to exclude the 0th column (_id)
        df = df.iloc[:, 1:]
                               
                               
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict("records")
        
        return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    # Return a scatter plot of df's data, setting breed as the x-axis and age as the y-axis.
    return [
        dcc.Graph(      
            figure = px.scatter(
                dff,
                x="breed",
                y="age_in_weeks_upon_outcome",
                color="sex_upon_outcome" # The color of the points is determined by the animals' sex.
            ).show()
        )
    ]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    return [
        # Center is given Austin coordinates
        dl.Map(style={"width": "1000px", "height": "500px"}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Make a marker with tool-tip and popup.
            dl.Marker(position=[30.75, -97.48], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1,9])
                ])
            ])
        ])
    ]
    



app